In [47]:
import pandas as pd
import numpy as np
from scipy import spatial
from sklearn.preprocessing import normalize
from scipy import sparse
from pyunlocbox import functions, solvers
from scipy import spatial
from matplotlib import pyplot as plt
import networkx as nx

In [48]:
df = pd.read_csv('processed.csv')
df.head()

,Unnamed: 0,Release date,Max resolution,Low resolution,Effective pixels,Zoom wide (W),Zoom tele (T),Normal focus range,Macro focus range,Storage included,Weight (inc. batteries),Dimensions,Price
0,0,1997.0,1024.0,640.0,0.0,NaN,114.0,70.0,40.0,4.0,420.0,95.0,179.0
1,1,1998.0,1280.0,640.0,NaN,38.0,114.0,50.0,0.0,4.0,NaN,158.0,NaN
2,2,2000.0,640.0,0.0,0.0,45.0,45.0,0.0,NaN,2.0,0.0,0.0,179.0
3,3,1999.0,NaN,640.0,0.0,NaN,35.0,0.0,0.0,4.0,0.0,0.0,269.0
4,4,1999.0,1152.0,640.0,0.0,43.0,NaN,50.0,0.0,40.0,300.0,128.0,1299.0


# K- NN Imputation

We are making use of k-NN imputation as discussed in the answer script

In [49]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=10, weights='distance', metric='nan_euclidean')
# fit on the dataset
imputer.fit(df)
# transform the dataset
X =imputer.transform(df)

In [50]:
from sklearn.preprocessing import normalize
#X = df.to_numpy()
X_norm = normalize(X, axis=0, norm='max')
#X_norm

#print(X_norm)

## Graph Learning algo

Implementation of the paper : Kalofolias, V. ["How to learn a graph from smooth signals"](https://arxiv.org/abs/1601.02513), AISTATS, 2016

Logic credits : https://github.com/rodrigo-pena/graph-learning which is under MIT License

In [51]:
def mapping(n):
    #Total number of edges
    ne = int(n*(n-1)/2)
    r1 = np.zeros((ne, ))
    r2 = np.zeros((ne, ))
    itr = 0
    for i in np.arange(1, n):
        r1[itr: (itr + (n - i))] = i - 1
        r2[itr: (itr + (n - i))] = np.arange(i, n)
        itr = itr + n - i
    row = np.concatenate((r1, r2))
    col = np.concatenate((np.arange(0, ne), np.arange(0, ne)))
    values = np.ones(len(row))
    K = sparse.coo_matrix((values, (row, col)), shape=(n, ne))
    return lambda w: K.dot(w), lambda d: K.transpose().dot(d)

In [52]:
def learngraph(X,alpha=1,beta=1,step=0.5,maxit=1000, rtol=1e-5):
    N = X.shape[0]
    z = spatial.distance.pdist(X,'euclidean') # calculating pairwise distance
    w0 = np.zeros(z.shape)
    K, Kt = mapping(N)
    norm_K = np.sqrt(2 * (N - 1))
    
    # Assemble functions in the objective
    f1 = functions.func()
    f1._eval = lambda w: 2 * np.dot(w, z)
    f1._prox = lambda w, gamma: np.maximum(0, w - (2 * gamma * z))

    f2 = functions.func()
    f2._eval = lambda w: - alpha * np.sum(np.log(np.maximum(
        np.finfo(np.float64).eps, K(w))))
    f2._prox = lambda d, gamma: np.maximum(
        0, 0.5 * (d + np.sqrt(d**2 + (4 * alpha * gamma))))

    f3 = functions.func()
    f3._eval = lambda w: beta * np.sum(w**2)
    f3._grad = lambda w: 2 * beta * w
    lipg = 2 * beta
    
    # Rescale stepsize
    stepsize = step / (1 + lipg + norm_K)

    # Solve problem
    solver = solvers.mlfbf(L=K, Lt=Kt, step=stepsize)
    problem = solvers.solve([f1, f2, f3], x0=w0, solver=solver, maxit=maxit,
                            rtol=rtol)

    # Transform weight matrix from vector form to matrix form
    W = spatial.distance.squareform(problem['sol'])
    W[W<0] = 0
    return W

In [53]:
beta=10
alpha = 1
W = learngraph(X_norm,beta = beta,alpha = 1)

Solution found after 264 iterations:
    objective function f(sol) = 1.443099e+02
    stopping criterion: RTOL


## Results

In [54]:
print(W.shape)
#print(np.around(W))
print(W)

(1038, 1038)
[[0.00000000e+00 6.99745129e-03 9.50931631e-08 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.99745129e-03 0.00000000e+00 1.00062299e-07 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [9.50931631e-08 1.00062299e-07 0.00000000e+00 ... 1.00427757e-07
  8.62809881e-08 1.00005867e-07]
 ...
 [0.00000000e+00 0.00000000e+00 1.00427757e-07 ... 0.00000000e+00
  2.70823200e-02 2.20528972e-02]
 [0.00000000e+00 0.00000000e+00 8.62809881e-08 ... 2.70823200e-02
  0.00000000e+00 9.67263219e-03]
 [0.00000000e+00 0.00000000e+00 1.00005867e-07 ... 2.20528972e-02
  9.67263219e-03 0.00000000e+00]]


In [55]:
A = np.zeros(W.shape)
for i in range(W.shape[0]):
       for j in range(W.shape[1]):
               if(W[i][j]!=0):
                    A[i][j]=1
               else:
                    A[i][j]=0
            
            
# Displaying graph nodes
G = nx.Graph()
for i in range(W.shape[0]):
    for j in range(W.shape[1]): 
       if A[i][j] == 1: 
          G.add_edge(i,j)
list(G.edges)

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 11),
 (0, 12),
 (0, 13),
 (0, 16),
 (0, 23),
 (0, 24),
 (0, 32),
 (0, 33),
 (0, 43),
 (0, 52),
 (0, 53),
 (0, 54),
 (0, 59),
 (0, 60),
 (0, 65),
 (0, 66),
 (0, 67),
 (0, 73),
 (0, 75),
 (0, 76),
 (0, 77),
 (0, 79),
 (0, 80),
 (0, 81),
 (0, 83),
 (0, 84),
 (0, 85),
 (0, 87),
 (0, 90),
 (0, 126),
 (0, 158),
 (0, 159),
 (0, 160),
 (0, 161),
 (0, 165),
 (0, 166),
 (0, 167),
 (0, 171),
 (0, 172),
 (0, 174),
 (0, 176),
 (0, 177),
 (0, 178),
 (0, 187),
 (0, 188),
 (0, 192),
 (0, 193),
 (0, 194),
 (0, 195),
 (0, 196),
 (0, 197),
 (0, 198),
 (0, 203),
 (0, 204),
 (0, 205),
 (0, 206),
 (0, 207),
 (0, 208),
 (0, 209),
 (0, 210),
 (0, 211),
 (0, 212),
 (0, 213),
 (0, 214),
 (0, 216),
 (0, 218),
 (0, 219),
 (0, 220),
 (0, 221),
 (0, 222),
 (0, 223),
 (0, 227),
 (0, 266),
 (0, 268),
 (0, 269),
 (0, 287),
 (0, 288),
 (0, 289),
 (0, 290),
 (0, 292),
 (0, 293),
 (0, 294),
 (0, 295),
 (0, 296),
 (0, 302),
 (0, 304),
 (